# Big Data Bowl 2024
I wanted to do more with this project: build a model to predict probability of tackling and whatnot, but it required too much time that I didn't have to spare. Therefore, I made a simpler data analysis project out of this, with a task of answering the question: *Do high-performing athletes in the NFL have a remarkably different physical build compared to the benchwarmers?*

## Hypothesis
I thought that there wouldn't be too much of a difference as scouts and the combine seem to look for certain traits amongst players when they are drafted. But I was hoping to be pleasantly surprised. 

## Plan
In this notebook I merge dataframes supplied by the NFL to calculate a contribution efficiency statistic by summing tackles, assists and fumbles divided by possible tackles. I then split the dataframe into 2 to look at the top half and bottom half of performers physically. I visualized the differences by position side by side with Seaborn and Matplotlib, then proceeded to do a hypothesis test between the two populations. 

## Extra
Finally I wanted to add in some clustering to try and partition the players, I decided to focus on cornerbacks as they were one of the most populated positions in this dataframe. I used an elbow plot to identify clusters, used KMeans and then plotted results showing results. I then used [PFF's grading system](https://www.pff.com/news/nfl-cornerback-rankings-final-sauce-gardner-retains-his-place-as-the-no-1-cornerback-in-the-nfl) to start a search for which cluster contained the *premier* corner backs in the league. This led me to some cool insights and a thought that one could use this data as a predictor to find rising stars at the position. 

# Creating a tackling metric

In order to compare players, we need to have a metric first to compare them.

In [ ]:
#Import necessary libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats 
import sklearn
from sklearn.cluster import KMeans


import warnings
warnings.filterwarnings("ignore") 

In [ ]:
tackles = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2024/tackles.csv") #Load dataframes
players = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2024/players.csv")
players_tackles = players.merge(tackles, on = 'nflId', how = "right") #Merge both dataframes

players_tackles[(players_tackles['tackle'] == 0) & (players_tackles['assist'] == 0) & (players_tackles['forcedFumble'] == 0) & (players_tackles['pff_missedTackle'] == 0)]
#Players either tackled, assisted on the tackle or forced a fumble OR they missed the tackle. I should add tackles, assist and forced fumbles, for tackle contributions, then add that number with missed tackles for total possible tackles

I use a right merge to collect defensive players, as they are the only ones included in the tackles df. The last line of code shows that players either contributed to a player or missed a tackle.

In [ ]:
players_tackles.head()

We have some redundant columns: nflId, birthDate and collegeName to name a few. There is also the issue of height being a string, we'll have to change that to a number variable later on. First we'll get to calculating a simple metric.

In [ ]:
players_tackles['tackle_contribution'] = players_tackles['tackle'] + players_tackles['assist'] + players_tackles['forcedFumble']
players_tackles['tackle_contribution'].value_counts()

What I've done is combine a player's tackles, assists and fumbles into one stat. 2 means they forced a fumble in addition to tackling or assisting on the tackle, 1 is just one of those actions and 0 means they missed the tackle. 

In [ ]:
players_tackles.drop(columns = ['tackle', 'assist','forcedFumble','birthDate', 'playId', 'gameId'], inplace = True)
players_tackles.reset_index(drop = True, inplace = True)
players_tackles

We drop columns that have already been accounted for or are irrelevant to cumulative statistics. 

We will now make two dataframes to count total tackles and missed tackles that we will merge later on to calculate total possible tackles. 

In [ ]:
missed_tackles = players_tackles.copy()
tackles_made = players_tackles.copy()

In [ ]:
tackles_made = tackles_made.groupby(["displayName", "nflId", "height", "weight","collegeName", "position"])[['tackle_contribution']].sum().reset_index()
tackles_made.at[322,'position'] = 'FS' 

missed_tackles = missed_tackles.groupby(["displayName", "nflId", "height", "weight","collegeName", "position"])[['pff_missedTackle']].sum().reset_index()
missed_tackles.at[322,'position'] = 'FS' 

I edit row 322 because Isiah Simmons was the only person listed as a DB within the dataframe. On ESPN he is listed as a safety, therefore I switched him to FS. 

In [ ]:
combined_tackles = tackles_made.merge(missed_tackles, on = ['displayName', 'nflId', 'height', 
                                                           'weight','collegeName', 'position'])

In [ ]:
combined_tackles['total_plays'] = combined_tackles['tackle_contribution'] + combined_tackles['pff_missedTackle'] #Calculate total plays
combined_tackles['efficiency %'] = round(combined_tackles['tackle_contribution'] / combined_tackles['total_plays'],2) * 100 #Calculate contribution

In [ ]:
combined_tackles['total_plays'].value_counts()

Almost 200 players have less than or equal to 10 plays total. Let's sort by the number of tackles and separate the dataframe into a top and bottom half, to view players that contribute more. Keep in mind there are 9 weeks of data here, so less than 10 is averaging slightly over 1 tackle a game, which is bad for every position - we don't need to segment the dataframe based off position then.

In [ ]:
sorted_df = combined_tackles.sort_values(by = ["total_plays", 'efficiency %'], ascending = False)
top_half = sorted_df.iloc[:400, :].reset_index(drop = True)
bot_half = sorted_df.iloc[400:, :].reset_index(drop = True)

We should now change the height to a numerical variable. 

In [ ]:
def calc_height(row):
    return (int(row['height'].split("-")[0])*12) + int(row['height'].split("-")[1])
top_half['height'] = top_half.apply(calc_height, axis = 1)
bot_half['height'] = bot_half.apply(calc_height, axis = 1)

We can do visual analysis with this dataframe now and see how weight and height can impact efficiency.

In [ ]:
defensive_positions = list(sorted_df['position'].unique()) #
defensive_positions

# Visualizing Physical Distributions

In [ ]:
grouped_high = top_half.groupby("position").size().reset_index().rename({0 : "Count"}, axis = 1).sort_values(by = "Count", ascending = False).reset_index(drop = True)
grouped_low = bot_half.groupby("position").size().reset_index().rename({0 : "Count"}, axis = 1).sort_values(by = "Count", ascending = False).reset_index(drop = True)
both = grouped_high.merge(grouped_low, on = "position", suffixes = ("_high", "_low"))
both['total'] = both['Count_high'] + both['Count_low']

In [ ]:
both.plot(x = "position", y = ["Count_high", "Count_low"], kind = "bar")
plt.title("Counts of Different Positions in top tacklers vs bottom tacklers")
plt.xticks(rotation = 45)
plt.show()

We can do a p-test on this to see if there's a significance to positions being separated. 

In [ ]:
def z_score(a, b):
    num = a-b
    den = (a+b)**(1/2)
    return num / den
both['p_value'] = scipy.stats.norm.sf(abs(both.apply(lambda x: z_score(x['Count_high'], x['Count_low']), axis = 1)))
both.sort_values(by = "p_value")

The clear difference in skill between top and bottom performers lies within the defensive end position, free safety also has a clear significance as well. Positions such as MLB and NT seem to be fairly spread across either dataframe, but we could've seen that without the significance test. 

## Weight Differences

In [ ]:
fig = plt.figure(figsize = (18,12))
plt.style.use("ggplot")
colors = ['firebrick', 'navy']
labels = ['Top Players', 'Bottom Players']
for idx, position in enumerate(defensive_positions):
    ax = fig.add_subplot(5,2,idx+1)
    top_df = top_half[top_half['position'] == position]
    bot_df = bot_half[bot_half['position'] == position]
    bins = int(int(top_df.shape[0] ** (1/2)) + int(bot_df.shape[0] ** (1/2))/2) #Take average of shapes, it's recommended that bins are the size of the df, square root of that and rounded up.
    plt.hist([top_df['weight'], bot_df['weight']], color = ['firebrick','navy'], bins = bins, label = ['Top Players', 'Bottom Players']) #Stacked histogram
    ax.legend()
    plt.title(f"Weight distribution top players vs bottoms players for {position.upper()}")
plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize = (18,12))
plt.style.use("ggplot")
labels = ['Top Players', 'Bottom Players']
for idx, position in enumerate(defensive_positions):
    ax = fig.add_subplot(5,2,idx+1)
    top_df = top_half[top_half['position'] == position]
    bot_df = bot_half[bot_half['position'] == position]
    sns.violinplot([top_df['weight'], bot_df['weight']]) 
    plt.title(f"Weight distribution top players vs bottoms players for {position.upper()}")
    plt.xticks(ticks = [0,1], labels = labels)
plt.tight_layout()
plt.show()

This data doesn't indicate much in terms of difference. For positions with too low of count, such as MLB or NT, the sample size isn't reliable. Whereas for the positions that do have enough people, the distribution is quite similar. You can make a case that ILB and DT have compact populations, with a smaller range of values, but still the mean weight is pretty much the same. 

### Height Differences

In [ ]:
fig = plt.figure(figsize = (18,12))
plt.style.use("ggplot")
colors = ['firebrick', 'navy']
labels = ['Top Players', 'Bottom Players']
for idx, position in enumerate(defensive_positions):
    ax = fig.add_subplot(5,2,idx+1)
    top_df = top_half[top_half['position'] == position]
    bot_df = bot_half[bot_half['position'] == position]
    bins = int(int(top_df.shape[0] ** (1/2)) + int(bot_df.shape[0] ** (1/2))/2) #Take average of shapes, it's recommended that bins are the size of the df, square root of that and rounded up.
    plt.hist([top_df['height'], bot_df['height']], color = colors, bins = bins, label = labels) #Side by side histogram
    ax.legend()
    plt.title(f"Height distribution top players vs bottoms players for {position.upper()}")
plt.tight_layout()
plt.show()

So far it looks like height is pretty uniform between top and bottom players. 

In [ ]:
fig = plt.figure(figsize = (18,12))
plt.style.use("ggplot")
colors = ['firebrick', 'navy']
labels = ['Top Players', 'Bottom Players']
for idx, position in enumerate(defensive_positions):
    ax = fig.add_subplot(5,2,idx+1)
    top_df = top_half[top_half['position'] == position]
    bot_df = bot_half[bot_half['position'] == position]
    sns.violinplot([top_df['height'], bot_df['height']]) #Stacked histogram
    ax.legend()
    plt.title(f"Height distribution top players vs bottoms players for {position.title()}")
    plt.xticks(ticks = [0,1], labels = labels)
plt.tight_layout()
plt.show()

Again I don't see much of a difference in distribution of heights, which sort of makes sense. Scouts look for similar profiles in people, physicality does make the image of the player, but skill makes that image come to life. The differences between these players is their experience, intuiton and skill ceiling. 

## P value between average heights and weights

In [ ]:
top_half_pos = top_half[['position', 'height', 'weight']]
bot_half_pos = bot_half[['position','height','weight']]

top_average = top_half_pos.groupby("position").mean().reset_index()
bot_average = bot_half_pos.groupby("position").mean().reset_index()
averages = top_average.merge(bot_average, on = "position", suffixes = ("_top", "_bot"))
averages[['position', 'height_top', 'height_bot', 'weight_top', 'weight_bot']]

In [ ]:
averages['z_score_height'] = averages.apply(lambda x: z_score(x['height_top'], x['height_bot']), axis = 1)
averages['z_score_weight'] = averages.apply(lambda x: z_score(x['weight_top'], x['weight_bot']), axis = 1)

averages['p_value_height'] = scipy.stats.norm.sf(abs(averages['z_score_height']))
averages['p_value_weight'] = scipy.stats.norm.sf(abs(averages['z_score_weight']))

In [ ]:
averages[['position', 'p_value_height', 'p_value_weight']]

We see that in terms of height almost all are .48-.49 in p value, which indicates no difference in population. Regarding weight, there is a bit more variation, we see the lowest p values in DT, MLB and NT, meaning these positions have the largest difference in average weight. 

Regardless of the reason for this, we can safely say there isn't a large difference in weights and heights of the most efficient and inefficient defensive players.

# Clustering

I wanted to add in some clustering in an attempt to identify segments of players. I thought I should focus on a single position to add some specificity, therefore I isolated cornerbacks since they had one of the highest counts of players across the dataframe. 

In [ ]:
combined_tackles

In [ ]:
cluster_data = combined_tackles[['displayName', 'height', 'weight', 'position', 'tackle_contribution', 'total_plays', 'efficiency %']]
cluster_data['height'] = cluster_data.apply(calc_height, axis = 1)

In [ ]:
corner_backs = cluster_data[cluster_data['position'] == "CB"].drop(columns = "position")

We can now do an elbow plot to identify how many clusters to use. This method is a heuristic that identifies the variance explained across an increasing number of clusters. It is called an "elbow" plot because we choose a k where the variance starts to level off and an increasing number of clusters is not needed to lower the variance by a significant amount. 

In [ ]:
wcss = [] 
for i in range(1, 11): 
    kmeans = KMeans(n_clusters = i, random_state = 42)
    kmeans.fit(corner_backs.iloc[:, 1:]) 
    wcss.append(kmeans.inertia_)
    
plt.plot(wcss)
plt.title("Elbow Plot for Corner Backs")
plt.show()

In [ ]:
kmeans = KMeans(n_clusters = 3, random_state = 42)
y_kmeans = kmeans.fit_predict(corner_backs.iloc[:, 1:])
corner_backs['cluster'] = y_kmeans

In [ ]:
fig = plt.figure(figsize = (18,12))
np.random.seed(0)
for idx, column in enumerate(list(corner_backs.columns)[1:]):
    if column == "cluster" or column == "efficiency %":
        continue
    ax = fig.add_subplot(2,2,idx+1)
    sns.scatterplot(data = corner_backs, x = column, y = 'efficiency %', hue = "cluster", palette = "Set1")
    plt.xlabel(column)
    plt.ylabel("efficiency %")
    plt.title(f"Cluster Distribution for Corner Backs")
    plt.legend()
plt.tight_layout()
plt.show()

We see that the green cluster seems to be the most played corner backs with moderate to high efficiency. These are the players that get targeted frequently, play a lot or get heavily involved and do pretty well. The red cluster is the group of players that play less, get targeted less or aren't as competitive and have subpar efficiency. The blue group is those that get targeted less, maybe contribute less, but they are very efficient when needed. * Notice that the contribution % is higher than 100 since plays could count as a tackle and a fumble, I think this is fair since it shows they are contributing more than just a tackle. 

When it comes to height and weight, it seems that every cluster has a wide spread of heights and weights, which reinforces my point that there isn't a clear distinction across nfl players in terms of weights and heights. 

Now I'll look at what players are in these clusters and see if they match what I just said in the previous text box. 

Top players we'll be looking for, according to [pff for 2022](https://www.pff.com/news/nfl-final-2022-cornerback-rankings) are:
- Sauce Gardner
- Patrick Surtain
- James Bradberry
- Jaire Alexander
- Stephon Gilmore
- Tyson Campbell
- Patrick Peterson
- Charvarius Ward
- Jalen Ramsey
- DJ Reed

In [ ]:
for player in ['Ahmad Gardner', 'Patrick Surtain', 'James Bradberry', 'Jaire Alexander', 'Stephon Gilmore', 'Tyson Campbell', 'Patrick Peterson', 'Charvarius Ward', 'Jalen Ramsey', 'D.J. Reed']:
    display(corner_backs[corner_backs['displayName'] == player])

As suspected, cluster 1 seems to be the group with majority of the premier corner backs in the league. Let's look at that cluster to identify some names that aren't in the pff link. These could be players that had a stellar season or may be a rising consistent star. 

In [ ]:
cluster_1 = corner_backs[corner_backs['cluster'] == 1].sort_values(by = ['total_plays', 'efficiency %'], ascending = False).reset_index(drop = True)
cluster_1.head(32)

I filtered by more than 30 possible tackles to target players that get more action and are still performing at a high level. 

Pff lists the top 32 corner backs in the league each year, so I displayed the top 32. Out of 32 players here, 13 aren't on either 2022's pff nor [2023 pff](https://www.pff.com/news/nfl-cornerback-rankings-final-sauce-gardner-retains-his-place-as-the-no-1-cornerback-in-the-nfl) (I didn't include Chidobe because Pff mentioned he would be ranked if he had more plays, he was injured in 2022): 
- Kyler Gordon (2nd year)
- Coby Bryant (2nd year)
- Jeff Okudah (4th year) 
- Cameron Dantzler (4th year) 
- Marco Wilson (3rd year) 
- Anthony Brown (8th year) 
- Kristian Fulton (4th year)
- Nate Hobbs (3rd year) 
- Chidobe Awuzie \* (7th year) 
- Michael Jackson (5th year) 
- Carlton Davis (6th year)
- Kaiir Elam (2nd year)
- Byron Murphy (5th year) 
- Amani Oruwariye (5th year)

\* means they were an honorable mention

Therefore 19 out of 32, almost 60% of players here are among the top cornerbacks in the league.

Also, given that corner backs typically last [only 3 years in the league](https://www.statista.com/statistics/240102/average-player-career-length-in-the-national-football-league/) we see the players that aren't on pff could be considered exceptional as well, since out of 11 out of 13 have been in the league for at least 3 years. The two players who aren't in this: **Kyler Gordon, Coby Bryant**; could be some names to look out for in the next few years. 

Also, there are **6** players not listed on 2022's Pff but do exist on the 2023 Pff list. Meaning, this method could be useful in targeting players that have potential to be amongst the top corner backs in the league:
- Roger McCreary (4th year) 
- Kenny Moore (7th year) (1x Pro-Bowl)
- Mike Hilton (7th year)
- Derek Stingley (2nd year)
- Donte Jackson (6th year)
- Paulson Adebo (3rd year)

The dimensions across the clusters is a bit large in difference:
- Cluster 0: 44
- Cluster 1: 82
- Cluster 2: 50

So let's try using 4 clusters to see if there's any more segmentation and balanced cluster dimensions.

In [ ]:
corner_backs_2 = cluster_data[cluster_data['position'] == "CB"].drop(columns = "position")

kmeans = KMeans(n_clusters = 4, random_state = 42)
y_kmeans = kmeans.fit_predict(corner_backs.iloc[:, 1:])
corner_backs_2['cluster'] = y_kmeans

In [ ]:
fig = plt.figure(figsize = (18,12))
np.random.seed(0)
for idx, column in enumerate(list(corner_backs_2.columns)[1:]):
    if column == "cluster" or column == "efficiency %":
        continue
    ax = fig.add_subplot(2,2,idx+1)
    sns.scatterplot(data = corner_backs_2, x = column, y = 'efficiency %', hue = "cluster", palette = "Set1")
    plt.xlabel(column)
    plt.ylabel("efficiency %")
    plt.title(f"Cluster Distribution for Corner Backs")
    plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
for player in ['Ahmad Gardner', 'Patrick Surtain', 'James Bradberry', 'Jaire Alexander', 'Stephon Gilmore', 'Tyson Campbell', 'Patrick Peterson', 'Charvarius Ward', 'Jalen Ramsey', 'D.J. Reed']:
    display(corner_backs_2[corner_backs_2['displayName'] == player])

In [ ]:
cluster_0 = corner_backs_2[corner_backs_2['cluster'] == 0].sort_values(by = ['total_plays', 'efficiency %'], ascending = False).reset_index(drop = True)
cluster_0.tail(10)

In [ ]:
corner_backs_2[corner_backs_2['cluster'] == 0].shape

It seems like we see a lot of familiar names across the PFF list, but I won't go in depth into this list as I did with the other. I do think this is. The dimensions across clusters are as follows (maybe a bit more balanced?):
- Cluster 0: 45 players
- Cluster 1: 46 players
- Cluster 2: 27 players
- Cluster 3: 58 players

# Conclusion
The question we were attempting to answer with this data was if there was a physical difference, in terms of heights and weights, across the top defensive performers and bottom performers per position. We saw there was no significant difference in physical heights and weights across positions but there was a significance in number of players in top efficiency vs bottom efficiency in certain positions. These positions were *defensive end* and *free safety*. 

I then did clustering for a bit of fun and found that it could be used to identify corner backs that are in the same realm as the top players at that position. This could be a fun thing to check up on each year and see if it's accurate. 
    